In [1]:
# import preprocessing tools
from sklearn import preprocessing
import numpy as np
import pandas as pd
import pickle

# import learning/evaluation
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

# typing
from typing import Dict, List, Callable, Any, Tuple, Optional, \
    Counter as CounterType, Set

# prepare data
from data import *

In [2]:
def make_target(df, h):
    y = df.shift(periods=-h, freq='1H')["pc"]
    df.pop('pc15')
    df.pop('pc30')
    df.pop('pc45')
    y = y[h:]
    X = df[:-h]
    return X, y

In [22]:
# run a series of experiments

horizons = [1, 3, 6, 12, 24, 36]
# sensors = [312, 386, 235, 368, 290, 1]
#horizons = [1]
#sensors = [312]
sensors = [1, 24, 35, 43, 44, 68, 103, 105, 106, 108, 109, 110, 116, 117, 122, 124, 144, 145, 157, 168, 177,
 179, 223, 225, 236, 239, 243, 245, 249, 253, 257, 259, 262, 285, 286, 290, 293, 301, 303, 312, 333, 358, 359,
 362, 365, 367, 377, 378, 379, 382, 384, 385, 388, 389, 391, 392, 394, 395]


features = []

for s in sensors:
    for h in horizons:
        output = "s{}h{}".format(s, h)
        print("\n\n *** {} *** \n".format(output))

        output_folder_name = output

        # check evaluation
        # read log from last generation        
        object = pd.read_pickle(f'log/{output_folder_name}/generation_1000.pickle')

        # list of best-scoring EvalItem objects for each number of features
        best = list(object.front.values())
        
        # load data
        static_df = load_static()
        weather_df = pd.read_pickle('data/weather/weather.pkl')
        # load sensor - last parameter is id
        final_df = make_final(static_df, weather_df, s)

        X_df, y_df = make_target(final_df, h)

        # basic dataset split
        n_sample = X_df.shape[0]
        n_test = int(n_sample * 0.8)

        labels_train = y_df.to_numpy()[:n_test]
        labels_test = y_df.to_numpy()[n_test:]

        XX_train = X_df.to_numpy()[:n_test, :]
        XX_test = X_df.to_numpy()[n_test:, :]
        
        sensor_features = []

        for item in best:
            # names of best features
            selected_features = final_df.iloc[:, item.genes].columns.tolist()
            #selected_features.pop()
            #selected_features.append('pc')

            X = X_df[selected_features].values.astype(float)
            y = y_df.values.astype(float)

            # evaluates each set of features with cross validation
            model = DecisionTreeRegressor()
            cvs = cross_val_score(model, X, y, cv=10)
            
            sensor_features.append({
                "sensor": s,
                "horizon": h,
                "accuracy": cvs.mean(),
                "stdev": cvs.std(),
                "features": selected_features,
            })
            
            print("Features:", selected_features)
            print("Accuracy: ", cvs.mean(), " stdev: ", cvs.std(), "\n")
        
        features.append(sensor_features)



 *** s1h1 *** 

Features: ['pc']
Accuracy:  0.7463628087408132  stdev:  0.03433775720257195 

Features: ['temperature25', 'windBearing44', 'pc', 'timeOfDay', 'weekEnd']
Accuracy:  0.7678963255584579  stdev:  0.0376409341684072 

Features: ['pc', 'timeOfDay']
Accuracy:  0.7914004702063769  stdev:  0.055885207811422984 

Features: ['temperature25', 'pc', 'timeOfDay', 'weekEnd']
Accuracy:  0.7742713196477128  stdev:  0.04076370506981909 

Features: ['temperature3', 'temperature33', 'pc', 'pc_ma_1H_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.7989307150095675  stdev:  0.039055928315935316 

Features: ['temperature4', 'dewPoint12', 'visibility31', 'temperature33', 'pc', 'pc_ma_1H_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.8082505460665615  stdev:  0.036015125690039194 

Features: ['temperature4', 'dewPoint12', 'humidity14', 'visibility31', 'temperature33', 'pc', 'pc_ma_1H_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.7917809979268271  stdev:  0.04545159707428145 

Features: ['temperature25'

Features: ['temperature26', 'i3', 'pc_ma_1H_3D', 'pc_ma_7D_2H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6048296466827203  stdev:  0.27926808652942886 

Features: ['i3', 'pc_ma_7D', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6152896707363593  stdev:  0.20057079617807758 

Features: ['dewPoint47', 'i3', 'pc_ma_7D_2H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6178172910129449  stdev:  0.15215398564806604 

Features: ['temperature26', 'i3', 'pc_std_30D', 'pc_ma_1H_3D', 'pc_ma_7D_2H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6396076244731319  stdev:  0.20628626558535307 

Features: ['temperature47', 'i3', 'pc_std_30D', 'pc_ma_1H_3D', 'pc_ma_6H_3D', 'pc_ma_7D_2H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.5647392694081309  stdev:  0.36218890403324455 

Features: ['cloudCover34', 'temperature47', 'i3', 'pc_std_30D', 'pc_ma_1H_3D', 'pc_ma_6H_3D', 'pc_ma_7D_2H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.5737080897552287  stdev:  0.3136151478435067 

Features: ['windBearing5', 'temperature26', 'temperature47',

Features: ['temperature28', 'pc_ma_1H', 'pc_std_7D', 'pc_ma_1D_2H', 'timeOfDay', 'dayOfWeek', 'holiday']
Accuracy:  0.8256726925457245  stdev:  0.11521421035311247 

Features: ['pressure0', 'temperature3', 'pc_ma_7D_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.8262188928695962  stdev:  0.06583617868212668 

Features: ['pressure0', 'temperature3', 'pressure5', 'pc_ma_7D_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.8167128021738309  stdev:  0.09189768170286262 

Features: ['dewPoint1', 'pc_ma_7D_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.8369614537236183  stdev:  0.06418824035397071 

Features: ['windSpeed11', 'temperature28', 'pc_ma_1H', 'pc_std_7D', 'pc_ma_1D_2H', 'timeOfDay', 'dayOfWeek', 'holiday']
Accuracy:  0.8218355347148233  stdev:  0.10490922951757954 



 *** s35h36 *** 

Features: ['timeOfDay']
Accuracy:  0.6002880111203551  stdev:  0.04168481045624357 

Features: ['pc', 'timeOfDay', 'dayBeforeHoliday', 'dayOfYear', 'dayOfWeek']
Accuracy:  0.8773675529346574  stdev:  0.05865256

Features: ['dewPoint31', 'visibility34', 'windBearing44', 'pc', 'pc_ma_6H', 'pc_ma_7D_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.9101178703186503  stdev:  0.10873085879876852 

Features: ['cloudCover18', 'humidity22', 'temperature29', 'pressure29', 'dewPoint31', 'visibility34', 'windBearing44', 'pc', 'pc_ma_6H', 'pc_ma_7D_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.9073669552261295  stdev:  0.09645325927014248 



 *** s44h3 *** 

Features: ['timeOfDay']
Accuracy:  0.5150608327313372  stdev:  0.2787891402455759 

Features: ['temperature0', 'pc_std_1D', 'timeOfDay', 'weekEnd']
Accuracy:  0.8422510055318766  stdev:  0.2865452399542682 

Features: ['temperature0', 'temperature16', 'pc_std_1D', 'timeOfDay', 'weekEnd']
Accuracy:  0.8314296447495341  stdev:  0.30863267796193805 

Features: ['timeOfDay', 'weekEnd']
Accuracy:  0.8050941937153686  stdev:  0.3530139713872456 

Features: ['temperature0', 'temperature6', 'pressure34', 'pc_std_1D', 'timeOfDay', 'weekEnd']
Accuracy:  0.842647503455

Features: ['temperature9', 'temperature14', 'pressure16', 'temperature20', 'windBearing34', 'pc', 'pc_ma_1D', 'pc_ma_1D_6H', 'timeOfDay', 'weekEnd']
Accuracy:  0.48486395251658393  stdev:  0.16918350011442948 



 *** s68h6 *** 

Features: ['timeOfDay']
Accuracy:  -0.09322026368341987  stdev:  0.4040591313536071 

Features: ['timeOfDay', 'dayOfWeek']
Accuracy:  0.1212982996644724  stdev:  0.46895181707965383 

Features: ['timeOfDay', 'dayOfWeek', 'holiday']
Accuracy:  0.11833081728590437  stdev:  0.4954448867707619 



 *** s68h12 *** 

Features: ['pc_ma_1H_12H']
Accuracy:  0.22816736878055507  stdev:  0.0691063179278028 



 *** s68h24 *** 

Features: ['i2']
Accuracy:  0.15979983091383004  stdev:  0.15444124306703202 

Features: ['i2', 'dayBeforeHoliday']
Accuracy:  0.16542159776196824  stdev:  0.1563771553114844 



 *** s68h36 *** 

Features: ['pc_ma_1H_12H']
Accuracy:  -0.18153238634486413  stdev:  0.11445408294456004 



 *** s103h1 *** 

Features: ['timeOfDay']
Accuracy:  0.47049

...

Features: ['i2', 'pc_ma_6H', 'pc_std_6H', 'pc_ma_1H_1H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6980467884753865  stdev:  0.05457565530631192 

Features: ['i2', 'pc_ma_1H_1H', 'timeOfDay', 'weekEnd']
Accuracy:  0.6537498515820304  stdev:  0.05966160293227012 

Features: ['cloudCover20', 'i2', 'pc_ma_1H_1H', 'timeOfDay', 'weekEnd']
Accuracy:  0.655985422506243  stdev:  0.06314955469942146 

Features: ['i2', 'pc_ma_6H', 'pc_std_6H', 'pc_ma_1H_1H', 'pc_ma_1H_3H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6915012676946879  stdev:  0.04796981403331814 

Features: ['pressure0', 'temperature3', 'dewPoint32', 'windBearing46', 'i2', 'pc_ma_1H_1H', 'timeOfDay', 'weekEnd']
Accuracy:  0.6165323910078936  stdev:  0.09197414097530052 



 *** s382h6 *** 

Features: ['weekEnd']
Accuracy:  0.19976443359354726  stdev:  0.26974279561870895 

Features: ['i1', 'timeOfDay', 'weekEnd']
Accuracy:  0.5726150231088927  stdev:  0.07508209396857686 

Features: ['temperature21', 'i1', 'pc_ma_1H_1H', 'pc_ma_1H_2H', '

Features: ['timeOfDay']
Accuracy:  0.42778023841657575  stdev:  0.22016739219749867 

Features: ['pc_ma_30D_1D', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.46391410804328803  stdev:  0.20239374467372037 

Features: ['pc_ma_7D', 'pc_ma_30D_2D', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6480255925185666  stdev:  0.08908156200696894 

Features: ['timeOfDay', 'dayOfWeek']
Accuracy:  0.5603614368902881  stdev:  0.23054193620034466 

Features: ['pc_ma_7D', 'pc_ma_7D_7D', 'pc_ma_30D_2D', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6338173983870302  stdev:  0.10927916861970334 

Features: ['cloudCover19', 'humidity32', 'pc_ma_7D', 'pc_ma_30D_2D', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6441260441958968  stdev:  0.10777522760620734 



 *** s385h6 *** 

Features: ['timeOfDay']
Accuracy:  0.42720163874454214  stdev:  0.22144501549253223 

Features: ['pc_ma_7D', 'pc_ma_30D_2D', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.6461382933308758  stdev:  0.11556387047883383 

Features: ['pc_ma_30D_1H', 'timeOfDay', 'dayOf

Features: ['temperature2', 'temperature13', 'dewPoint22', 'pressure24', 'dewPoint36', 'temperature38', 'pc_ma_1D', 'pc_ma_1D_6H', 'timeOfDay', 'dayOfYear']
Accuracy:  0.5176596105755544  stdev:  0.12140812226984649 

Features: ['temperature2', 'temperature13', 'dewPoint22', 'pressure24', 'dewPoint36', 'temperature38', 'pressure39', 'pc_ma_1D', 'pc_ma_1D_6H', 'timeOfDay', 'dayOfYear']
Accuracy:  0.5212513251951649  stdev:  0.14274703387374027 



 *** s388h24 *** 

Features: ['pc_ma_1H']
Accuracy:  0.5636915387226731  stdev:  0.12677153449144765 

Features: ['temperature0', 'temperature14', 'temperature16', 'dewPoint17', 'pressure26', 'pc', 'pc_ma_1H', 'pc_ma_1H_2D', 'pc_ma_6H_3H', 'dayOfYear']
Accuracy:  0.41553419813188686  stdev:  0.25199917865375726 

Features: ['temperature0', 'dewPoint17', 'pc', 'pc_ma_1H', 'pc_ma_1H_2D', 'pc_ma_6H_3H', 'dayOfYear']
Accuracy:  0.44433582443789954  stdev:  0.1952301721067866 

Features: ['temperature0', 'dewPoint2', 'temperature11', 'temperature14'

Features: ['pc_ma_1H', 'pc_ma_1D_7D', 'pc_ma_7D_3D', 'pc_ma_30D_6H', 'timeOfDay']
Accuracy:  0.6899921125826024  stdev:  0.07476582473524179 

Features: ['windSpeed42', 'pc_ma_1H', 'pc_ma_7D_3D', 'pc_ma_30D_6H', 'timeOfDay', 'dayOfWeek']
Accuracy:  0.7017401804447825  stdev:  0.11356924404536783 



 *** s391h3 *** 

Features: ['pc_ma_1H']
Accuracy:  0.3752025346376376  stdev:  0.052239543705778414 

Features: ['pc_ma_1H', 'timeOfDay']
Accuracy:  0.49296271140719333  stdev:  0.11234645754077503 

Features: ['pc_ma_1H', 'timeOfDay', 'weekEnd']
Accuracy:  0.6085782786080512  stdev:  0.12973458873906688 

Features: ['pc_ma_1H', 'pc_ma_7D_2H', 'timeOfDay', 'weekEnd']
Accuracy:  0.6214026524261025  stdev:  0.16715375611123579 

Features: ['pc', 'pc_ma_1H', 'pc_ma_7D_2H', 'timeOfDay', 'weekEnd']
Accuracy:  0.6351108734346381  stdev:  0.15968959074428415 

Features: ['dewPoint36', 'pc', 'pc_ma_1H', 'pc_ma_7D_2H', 'timeOfDay', 'weekEnd']
Accuracy:  0.6364768651611751  stdev:  0.140505742887530

Features: ['pc_ma_1H', 'pc_ma_1H_1D', 'pc_ma_1D_3H', 'pc_ma_1D_2D', 'timeOfDay', 'dayBeforeHoliday', 'dayOfWeek']
Accuracy:  0.8126079184277746  stdev:  0.04151172176837322 

Features: ['pc_ma_1H', 'pc_ma_1H_1D', 'pc_ma_1D_2H', 'pc_ma_1D_3H', 'pc_ma_1D_2D', 'timeOfDay', 'dayBeforeHoliday', 'dayOfWeek']
Accuracy:  0.815863117859702  stdev:  0.04341172838956938 



 *** s392h36 *** 

Features: ['timeOfDay']
Accuracy:  0.6511868835354395  stdev:  0.08442960205548769 

Features: ['timeOfDay', 'dayOfWeek', 'monthOfYear']
Accuracy:  0.8489814815901434  stdev:  0.05677846114656304 

Features: ['timeOfDay', 'monthOfYear']
Accuracy:  0.7544534035869737  stdev:  0.05552142265811434 



 *** s394h1 *** 

Features: ['pc']
Accuracy:  0.6841858200278148  stdev:  0.028892472607134494 

Features: ['timeOfDay', 'dayOfYear']
Accuracy:  0.9407189340569284  stdev:  0.03426629705417203 

Features: ['windSpeed8', 'pc', 'timeOfDay', 'dayOfYear']
Accuracy:  0.9406544345375307  stdev:  0.03547346326029445 

Fe

In [4]:
# find a list of viable sensors
selected_features = ['temperature24', 'pc', 'pc_ma_1H', 'pc_ma_6H_2H', 'pc_ma_6H_6H', 'timeOfDay', 'dayOfWeek', 'holiday']

# load data
static_df = load_static()
weather_df = pd.read_pickle('data/weather/weather.pkl')

sensors = [1,
 24,
 35,
 43,
 44,
 68,
 79,
 100,
 103,
 105,
 106,
 108,
 109,
 110,
 116,
 117,
 122,
 124,
 144,
 145,
 157,
 168,
 177,
 179,
 223,
 225,
 236,
 239,
 243,
 245,
 249,
 253,
 257,
 259,
 262,
 264,
 285,
 286,
 290,
 293,
 301,
 303,
 312,
 333,
 358,
 359,
 361,
 362,
 365,
 367,
 369,
 377,
 378,
 379,
 382,
 384,
 385,
 388,
 389,
 391,
 392,
 394,
 395]

sfiltered = []

for s in sensors:
    for h in [24]:        
        # load sensor - last parameter is id
        final_df = make_final(static_df, weather_df, s)

        X_df, y_df = make_target(final_df, h)
        
        X = X_df[selected_features].values.astype(float)
        y = y_df.values.astype(float)

        # evaluates each set of features with cross validation
        model = DecisionTreeRegressor()
        #model = LinearRegression()
        try:
            cvs = cross_val_score(model, X, y, cv=10)        
            print("S", s, "Accuracy: ", cvs.mean(), " stdev: ", cvs.std(), "\n")
            if cvs.mean() > 0.25 and cvs.mean() < 1.0:
                sfiltered.append(s)
        except:
            pass
        
        


S 1 Accuracy:  0.4347986586153034  stdev:  0.34845786131596995 

S 24 Accuracy:  0.5339004609053464  stdev:  0.3827544081437165 

S 35 Accuracy:  0.8360211703523828  stdev:  0.11271484650155704 

S 43 Accuracy:  0.40648926786936884  stdev:  0.07934755601434317 

S 44 Accuracy:  0.9049711948239565  stdev:  0.06742841239589625 

S 68 Accuracy:  0.3983356099755394  stdev:  0.12288704521274875 

S 79 Accuracy:  1.0  stdev:  0.0 

S 100 Accuracy:  1.0  stdev:  0.0 

S 103 Accuracy:  0.5703501877137722  stdev:  0.07948094411120131 

S 105 Accuracy:  0.5936918792817274  stdev:  0.13163422022648805 

S 106 Accuracy:  0.3921781155501824  stdev:  0.14611016894835918 

S 108 Accuracy:  0.5719351190330915  stdev:  0.08033770369633118 

S 109 Accuracy:  0.2637369778861608  stdev:  0.17420678910445733 

S 110 Accuracy:  0.9011753305112642  stdev:  0.058110991880102957 

S 116 Accuracy:  0.34578432038068874  stdev:  0.17684123251648642 

S 117 Accuracy:  0.3909187591097076  stdev:  0.343735208671987 

[1,
 24,
 35,
 43,
 44,
 68,
 79,
 100,
 103,
 105,
 106,
 108,
 109,
 110,
 116,
 117,
 122,
 124,
 144,
 145,
 157,
 168,
 177,
 179,
 223,
 225,
 236,
 239,
 243,
 245,
 249,
 253,
 257,
 259,
 262,
 264,
 285,
 286,
 290,
 293,
 301,
 303,
 312,
 333,
 358,
 359,
 361,
 362,
 365,
 367,
 369,
 377,
 378,
 379,
 382,
 384,
 385,
 388,
 389,
 391,
 392,
 394,
 395]

In [15]:
viable_sensors = [1, 24, 35, 43, 44, 68, 103, 105, 106, 108, 109, 110, 116, 117, 122, 124, 144, 145, 157, 168, 177,
 179, 223, 225, 236, 239, 243, 245, 249, 253, 257, 259, 262, 285, 286, 290, 293, 301, 303, 312, 333, 358, 359, 362,
 365, 367, 377, 378, 379, 382, 384, 385, 388, 389, 391, 392, 394, 395]

In [26]:
features

[[{'sensor': 1,
   'horizon': 1,
   'accuracy': 0.7463628087408132,
   'stdev': 0.03433775720257195,
   'features': ['pc']},
  {'sensor': 1,
   'horizon': 1,
   'accuracy': 0.7678963255584579,
   'stdev': 0.0376409341684072,
   'features': ['temperature25',
    'windBearing44',
    'pc',
    'timeOfDay',
    'weekEnd']},
  {'sensor': 1,
   'horizon': 1,
   'accuracy': 0.7914004702063769,
   'stdev': 0.055885207811422984,
   'features': ['pc', 'timeOfDay']},
  {'sensor': 1,
   'horizon': 1,
   'accuracy': 0.7742713196477128,
   'stdev': 0.04076370506981909,
   'features': ['temperature25', 'pc', 'timeOfDay', 'weekEnd']},
  {'sensor': 1,
   'horizon': 1,
   'accuracy': 0.7989307150095675,
   'stdev': 0.039055928315935316,
   'features': ['temperature3',
    'temperature33',
    'pc',
    'pc_ma_1H_3H',
    'timeOfDay',
    'dayOfWeek']},
  {'sensor': 1,
   'horizon': 1,
   'accuracy': 0.8082505460665615,
   'stdev': 0.036015125690039194,
   'features': ['temperature4',
    'dewPoint12',


In [25]:
# save features
with open("results/features.pkl", "wb") as f:
    pickle.dump(features, f)

In [4]:
with open("results/features.pkl", "rb") as f:
    features = pickle.load(f)

In [7]:
# feature weights
fweights = {}

# find optimal features
for horizon in features:
    print(horizon)
    
    # normalize accuracies
    accuracies = []
    for candidate in horizon:
        accuracies.append(candidate["accuracy"])        
    a_max = max(accuracies)
    a_min = min(accuracies)
    if a_min > a_max - 0.2:
        a_min = a_max - 0.2
    k = 1 / (a_max - a_min)
    
    for i in range(len(accuracies)):        
        new_acc = (accuracies[i] - a_min) * k
        
    #print(candidate)
    for f in candidate["features"]:
        #print(f)
    
        if not f in fweights:
            fweights[f] = 0
        fweights[f] = fweights[f] + new_acc
        

[{'sensor': 1, 'horizon': 1, 'accuracy': 0.7463628087408132, 'stdev': 0.03433775720257195, 'features': ['pc']}, {'sensor': 1, 'horizon': 1, 'accuracy': 0.7678963255584579, 'stdev': 0.0376409341684072, 'features': ['temperature25', 'windBearing44', 'pc', 'timeOfDay', 'weekEnd']}, {'sensor': 1, 'horizon': 1, 'accuracy': 0.7914004702063769, 'stdev': 0.055885207811422984, 'features': ['pc', 'timeOfDay']}, {'sensor': 1, 'horizon': 1, 'accuracy': 0.7742713196477128, 'stdev': 0.04076370506981909, 'features': ['temperature25', 'pc', 'timeOfDay', 'weekEnd']}, {'sensor': 1, 'horizon': 1, 'accuracy': 0.7989307150095675, 'stdev': 0.039055928315935316, 'features': ['temperature3', 'temperature33', 'pc', 'pc_ma_1H_3H', 'timeOfDay', 'dayOfWeek']}, {'sensor': 1, 'horizon': 1, 'accuracy': 0.8082505460665615, 'stdev': 0.036015125690039194, 'features': ['temperature4', 'dewPoint12', 'visibility31', 'temperature33', 'pc', 'pc_ma_1H_3H', 'timeOfDay', 'dayOfWeek']}, {'sensor': 1, 'horizon': 1, 'accuracy': 0

In [6]:
fweights

{'temperature4': 5.072408465592733,
 'dewPoint12': 3.546154903241109,
 'humidity14': 2.3923799050696157,
 'dewPoint16': 1.8995485282709414,
 'visibility31': 2.921042813800427,
 'temperature33': 3.4035580442139026,
 'pc': 63.44765717259459,
 'pc_ma_1H_3H': 10.266257674083066,
 'timeOfDay': 256.56371250810145,
 'dayOfWeek': 174.73242368764332,
 'monthOfYear': 10.217368362627804,
 'weekEnd': 58.251020988950074,
 'holiday': 32.641178361681185,
 'pressure10': 1.4170799525692277,
 'humidity12': 0.4373231865169666,
 'visibility18': 2.3988735733583986,
 'dewPoint24': 2.1297279131013944,
 'visibility30': 0.4373231865169666,
 'dewPoint37': 1.9274361490020193,
 'humidity40': 0.4373231865169666,
 'windSpeed47': 3.247810643034984,
 'i3': 6.534555277356219,
 'pc_ma_7D_2D': 4.862529990383436,
 'dayOfYear': 33.28297348807057,
 'dayBeforeHoliday': 50.15066483673306,
 'windBearing2': 2.4577157144345496,
 'visibility42': 2.9132697989937624,
 'i1': 5.563935136812765,
 'pc_ma_1H_2H': 10.061070513027907,
 '